# Loan Application Orchestration Pipeline Demo (Agent Orchestration)

This notebook demonstrates the full loan application decision process **using the agent orchestration pipeline**. The orchestration function calls all tools and combines the results into a final summary.

In [1]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

# Import required modules and orchestration pipeline
from data_model import LoanApplicationJourney
from utils.config_loader import ConfigLoader
from orchestrator_pipeline import orchestrate_application
from tools.synthesize_summary import synthesize_summary
from agents import RunContextWrapper

# Import agent objects
from agents.fraud_agent import fraud_agent
from agents.sla_agent import sla_agent
from agents.recommendation_agent import get_recommendation
from agents.orchestrator_agent import orchestrator_agent
from agents.report_agent import report_agent
from agents.interest_rate_agent import interest_rate_agent

#### Test example

In [2]:
# Create a test LoanApplicationJourney instance
test_app = LoanApplicationJourney(
    application_id="APP-LOCAL-001",
    submitted_time="2025-06-01T09:00:00",
    reviewed_time="2025-06-01T09:30:00",
    approved_time="2025-06-01T10:00:00",
    rejected_time=None,
    processing_steps={"KYC": 72, "CreditCheck": 50, "FinalApproval": 35},
    flagged_for_fraud=False,
    monthly_income=50000,
    monthly_costs=1000,
    requested_amount=25000,
    monthly_debt=400
)

## Integrate and orchestrate the agents

In [5]:
# Create a python function that will orchestrate the deterministic workflow
fraud_result = fraud_agent.tools[0](RunContextWrapper(test_app))
sla_result = sla_agent.tools[0](RunContextWrapper(test_app))
interest_rate_result = interest_rate_agent.tools[0](RunContextWrapper(test_app))

print("Fraud Result:", fraud_result)
print("SLA Result:", sla_result)
print("Interest Rate Result:", interest_rate_result)

Fraud Result: {'flagged': False, 'label': 'No fraud indicators detected', 'risk_score': 0.0, 'triggered_rules': [], 'explanation': 'No indicators present'}
SLA Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}
Interest Rate Result: {'level': 'approve', 'label': 'Excellent'}


/Users/lucadicarlo/Desktop/My Projects/Oxford AI Summit/Loan-application-agent/agents/interest_rate_agent.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_rate = hist['Close'][-1]


In [ ]:
recommendation = get_recommendation(fraud_result, sla_result, interest_rate_result)
print("Recommendation:", recommendation)

Recommendation: **Recommendation: Reject the Loan Application**

While the fraud risk assessment indicates no indicators of fraud and the interest rate result is excellent, the SLA compliance has been violated due to issues in the KYC process. This violation raises concerns about the thoroughness of the application review. Therefore, it is advisable to reject the loan application until the KYC steps are completed in compliance with SLA requirements.


In [ ]:
# Example: Use interest_rate_agent to check for interest rate (sync call for demo)
interest_rate_result = interest_rate_agent.tools[0](RunContextWrapper(test_app))
print('Interest Rate Agent Result:', interest_rate_result)


In [ ]:
# Example: Use fraud_agent to check for fraud indicators (sync call for demo)
fraud_result = fraud_agent.tools[0](RunContextWrapper(test_app))
print('Fraud Agent Result:', fraud_result)

In [ ]:
# Example: Use sla_agent to check SLA compliance (sync call for demo)
sla_result = sla_agent.tools[0](RunContextWrapper(test_app))
print('SLA Agent Result:', sla_result)

In [ ]:
# Example: Use recommendation_agent to get a recommendation (sync call for demo)
rec_result = recommendation_agent.run('Provide recommendation', context=test_app)
print('Recommendation Agent Result:', getattr(rec_result, 'final_output', rec_result))

In [ ]:
# Example: Use orchestrator_agent to run the full orchestration (sync call for demo)
orc_result = orchestrator_agent.tools[0](RunContextWrapper(test_app))
print('Orchestrator Agent Result:', orc_result)

In [ ]:
# Example: Use report_agent to synthesize the summary (sync call for demo)
rep_result = report_agent.tools[0](RunContextWrapper(report))
print('Report Agent Result:', rep_result)

# TO DO:

## 1.Trend Analysis

### ToDo: Ask agents to analyze and take decisions based on the metrics below: 
####- interest rate cutoff : search the web to find what a reasonable interest rate is for a loan of this type -> 
####- fraud detection: based on the kyc and other metrics 
#- SLA compliance

### Goal: give AI more freedom to decide BY REMOVING THE CONFIG 

## 2.MCP Tools 
### Send email with report 